# Baseline: Задача №3 — поиск аномалий в заявках

Цель: без метки fraud выявить подозрительные заявки. Оценка качества — прокси: доля `Charged Off` среди топ‑аномалий на валидации/тесте.

In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)


In [19]:
df = pd.read_csv('financial_loan.csv')
print(df.shape)
df.head()


(38576, 24)


,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,loan_status,next_payment_date,member_id,purpose,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
0,1077430,GA,INDIVIDUAL,< 1 year,Ryder,C,RENT,11-02-2021,13-09-2021,13-04-2021,Charged Off,13-05-2021,1314167,car,C4,60 months,Source Verified,30000.0,0.0100,59.83,0.1527,2500,4,1009
1,1072053,CA,INDIVIDUAL,9 years,MKC Accounting,E,RENT,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1288686,car,E1,36 months,Source Verified,48000.0,0.0535,109.43,0.1864,3000,4,3939
2,1069243,CA,INDIVIDUAL,4 years,Chemat Technology Inc,C,RENT,05-01-2021,12-12-2021,09-01-2021,Charged Off,09-02-2021,1304116,car,C5,36 months,Not Verified,50000.0,0.2088,421.65,0.1596,12000,11,3522
3,1041756,TX,INDIVIDUAL,< 1 year,barnes distribution,B,MORTGAGE,25-02-2021,12-12-2021,12-03-2021,Fully Paid,12-04-2021,1272024,car,B2,60 months,Source Verified,42000.0,0.0540,97.06,0.1065,4500,9,4911
4,1068350,IL,INDIVIDUAL,10+ years,J&J Steel Inc,A,MORTGAGE,01-01-2021,14-12-2021,15-01-2021,Fully Paid,15-02-2021,1302971,car,A1,36 months,Verified,83000.0,0.0231,106.53,0.0603,3500,28,3835


## Признаки и исключения

In [20]:
# Заявочные признаки
features = [
    'annual_income', 'dti', 'loan_amount', 'term', 'purpose', 'home_ownership',
    'verification_status', 'address_state', 'application_type', 'emp_length', 'emp_title'
]

# Исключаем утечки/постфактум
exclude = [
    'total_payment', 'last_payment_date', 'next_payment_date',
    'last_credit_pull_date', 'issue_date', 'id', 'member_id'
]

work = df.copy()
work['term_months'] = work['term'].str.extract(r'(\d+)').astype(float)
features = [c for c in features if c != 'term'] + ['term_months']

X = work[features].copy()
y_proxy = (work['loan_status'] == 'Charged Off').astype(int)
print('features:', features)


features: ['annual_income', 'dti', 'loan_amount', 'purpose', 'home_ownership', 'verification_status', 'address_state', 'application_type', 'emp_length', 'emp_title', 'term_months']


## Train/Val/Test split

In [21]:
# Используем прокси-метку только для оценки
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_proxy, test_size=0.3, random_state=42, stratify=y_proxy
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print('train:', X_train.shape, 'val:', X_val.shape, 'test:', X_test.shape)


train: (27003, 11) val: (5786, 11) test: (5787, 11)


## Обработка emp_title (высокая кардинальность)

In [22]:
N = 25
vc = X_train['emp_title'].fillna('MISSING').value_counts()
keep = set(vc.head(N).index)

for part in [X_train, X_val, X_test]:
    part['emp_title'] = part['emp_title'].fillna('MISSING')
    part['emp_title'] = part['emp_title'].where(part['emp_title'].isin(keep), 'Other')


## Препроцессинг

In [23]:
cat_cols = [
    'purpose','home_ownership','verification_status','address_state',
    'application_type','emp_length','emp_title'
]
num_cols = ['annual_income','dti','loan_amount','term_months']

preprocess = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), num_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_cols)
    ]
)


## Модель 1: Isolation Forest

In [24]:
iso = IsolationForest(
    n_estimators=200,
    contamination=0.02,
    random_state=42
)

iso_clf = Pipeline(steps=[('prep', preprocess), ('model', iso)])
iso_clf.fit(X_train)

# score_samples: чем меньше, тем более аномально
iso_val_scores = -iso_clf.named_steps['model'].score_samples(
    iso_clf.named_steps['prep'].transform(X_val)
)
iso_test_scores = -iso_clf.named_steps['model'].score_samples(
    iso_clf.named_steps['prep'].transform(X_test)
)

iso_val_cut = np.quantile(iso_val_scores, 0.98)
iso_test_cut = np.quantile(iso_test_scores, 0.98)

iso_val_flag = (iso_val_scores >= iso_val_cut).astype(int)
iso_test_flag = (iso_test_scores >= iso_test_cut).astype(int)

print('val flagged:', iso_val_flag.mean())
print('test flagged:', iso_test_flag.mean())


val flagged: 0.020048392671966817
test flagged: 0.02004492828754104


## Модель 2: LOF (оценка на val/test)

In [25]:
# LOF работает только как fit_predict, поэтому обучаем на train и оцениваем отдельно
X_train_t = preprocess.fit_transform(X_train)
X_val_t = preprocess.transform(X_val)
X_test_t = preprocess.transform(X_test)

lof = LocalOutlierFactor(n_neighbors=35, contamination=0.02, novelty=True)
lof.fit(X_train_t)

lof_val_scores = -lof.score_samples(X_val_t)
lof_test_scores = -lof.score_samples(X_test_t)

lof_val_cut = np.quantile(lof_val_scores, 0.98)
lof_test_cut = np.quantile(lof_test_scores, 0.98)

lof_val_flag = (lof_val_scores >= lof_val_cut).astype(int)
lof_test_flag = (lof_test_scores >= lof_test_cut).astype(int)

print('val flagged (lof):', lof_val_flag.mean())
print('test flagged (lof):', lof_test_flag.mean())


val flagged (lof): 0.020048392671966817
test flagged (lof): 0.02004492828754104


## Прокси‑оценка: доля Charged Off среди топ‑аномалий

In [26]:
def charged_off_rate(y_true, flags):
    if flags.sum() == 0:
        return 0.0
    return float(y_true[flags == 1].mean())

# Isolation Forest
iso_val_rate = charged_off_rate(y_val.values, (val_scores >= np.quantile(val_scores, 0.98)).astype(int))
iso_test_rate = charged_off_rate(y_test.values, (test_scores >= np.quantile(test_scores, 0.98)).astype(int))

print('IsolationForest: Charged Off rate in anomalies')
print('val:', iso_val_rate)
print('test:', iso_test_rate)

# LOF
lof_val_rate = charged_off_rate(y_val.values, (val_scores >= np.quantile(val_scores, 0.98)).astype(int))
lof_test_rate = charged_off_rate(y_test.values, (test_scores >= np.quantile(test_scores, 0.98)).astype(int))

print('LOF: Charged Off rate in anomalies')
print('val:', lof_val_rate)
print('test:', lof_test_rate)

# Базовая доля Charged Off
print('overall Charged Off rate:', y_proxy.mean())


IsolationForest: Charged Off rate in anomalies
val: 0.09482758620689655
test: 0.15517241379310345
LOF: Charged Off rate in anomalies
val: 0.09482758620689655
test: 0.15517241379310345
overall Charged Off rate: 0.13824657818332642
